# Music Recommendation with KDB.AI

This example demonstrates how you can use KDB.AI to perform similarity recommendations using vector embeddings created from both categorical and numeric music data.

Applications like Spotify and YouTube Music perform hundreds of millions of song recommendations for users every single day. They do this by extracting a vast array of features about every given song and artist and comparing their characteristics.
By leveraging this sort of data, KDB.AI can be used to productionize a music recommendation system and help to quickly and efficiently find music similar to given input songs.

In this tutorial, we'll break down how you might perform similarity search on music, taking some Spotify data as an example and using KDB.AI as the vector database to store and query this data.
This breaks down as follows:

1. Read In Song Data
1. Create Vector Embeddings
1. Create KDB.AI Vector Database
1. Query Songs From Vector Database

## 1. Setup

### Import Packages

We will start by importing all of the Python packages needed to run this music recommendation system example.
This includes packages for reading in the data, embedding it as vectors, and interacting with the vector database.

In [1]:
%pip install -U gensim kdbai_client -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

In [3]:
# embedding categorical data
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec

In [4]:
# embedding numeric data
from sklearn.preprocessing import StandardScaler

In [34]:
# vector DB
import kdbai_client as kdbai
from getpass import getpass

In [6]:
# tokeniser
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/mwoods/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Configure Console

In order to fully view our embeddings when it comes to displaying the results, we must increase the maximum allowed column width in Pandas DataFrames from the default value.

In [7]:
pd.set_option("max_colwidth", 1000)

This removes a warning that appears when performing in-place column assignment.

In [8]:
pd.options.mode.chained_assignment = None

### Define Helper Functions

Defining these two helper functions will allow us to easily show the shape and head of any Pandas DataFrames or embedding arrays passed.

In [9]:
def show_df(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    return df.head()

In [11]:
def show_embeddings(embeddings: np.array) -> list[int]:
    print("Num Embeddings:", len(embeddings))
    print("Embedding Size:", len(embeddings[0]))
    return list(embeddings[0])

## 2. Read in Song Data

The song data we will read in will be taken from an [open-source Spotify dataset](https://www.kaggle.com/datasets/vatsalmavani/spotify-dataset) on Kaggle. There are 5 files on Kaggle, however, only one file is relevant to this analysis.
This dataset contains a list of metadata on 170,000 songs from 1921 to 2020. This metadata includes:
- Song Name
- Artist Name
- Song Year
- Various features about the song's music, including:
    * acousticness
    * danceability
    * duration_ms
    * energy
    * explicit
    * instrumentalness
    * key
    * liveness
    * loudness
    * mode
    * popularity
    * release_date
    * speechiness
    * tempo
    * valence
   

### Read Data from CSV

We can read this song data from a CSV into a Pandas DataFrame and show the resulting table.

In [12]:
raw_song_df = pd.read_csv("data/song_data.csv")

In [13]:
show_df(raw_song_df)

(170653, 19)


,id,name,artists,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,release_date,speechiness,tempo,valence,year
0,4BJqT0PrAfrxzMOxytFOIz,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","['Sergei Rachmaninoff', 'James Levine', 'Berliner Philharmoniker']",0.982,0.279,831667,0.211,0,0.878000,10,0.665,-20.096,1,4,1921,0.0366,80.954,0.0594,1921
1,7xPhfUan2yNtyFG0cUWkt8,Clancy Lowered the Boom,['Dennis Day'],0.732,0.819,180533,0.341,0,0.000000,7,0.160,-12.441,1,5,1921,0.4150,60.936,0.9630,1921
2,1o6I8BglA6ylDMrIELygv1,Gati Bali,['KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat'],0.961,0.328,500062,0.166,0,0.913000,3,0.101,-14.850,1,5,1921,0.0339,110.339,0.0394,1921
3,3ftBPsC5vPBKxYSee08FDH,Danny Boy,['Frank Parker'],0.967,0.275,210000,0.309,0,0.000028,5,0.381,-9.316,1,3,1921,0.0354,100.109,0.1650,1921
4,4d6HGyGT8e121BsdKmw9v6,When Irish Eyes Are Smiling,['Phil Regan'],0.957,0.418,166693,0.193,0,0.000002,3,0.229,-10.096,1,2,1921,0.0380,101.665,0.2530,1921


### Pre-process Data

Here we will perform a few operations on this Pandas DataFrame to get it into the correct format for creating the vector embeddings for our vector database.
This will include:
- Adding a column prefix
- Removing excess columns
- Fixing column values
- Combining columns into one
- Removing duplicate rows

Once these pre-processing steps have been carried out, our data will be clean and in the correct format to start creating embeddings.

In [14]:
# add "song_" prefix to col names
song_df = raw_song_df.add_prefix("song_")

In [15]:
# drop unused cols
song_df = song_df.drop(columns=["song_id", "song_release_date"])

In [16]:
# fix artists list names - remove quotes
def fix_artists(str_list):
    return ", ".join([v for v in str_list.rstrip("']").lstrip("['").split("', '")])


song_df["song_artists"] = song_df["song_artists"].apply(fix_artists)

In [17]:
# combine song_name & song_artists into song_description
song_df.insert(
    0, "song_description", song_df["song_name"] + " - " + song_df["song_artists"]
)

In [18]:
# remove duplicate rows
song_data = song_df[
    ~song_df.duplicated(subset=["song_description"], keep="first")
].reset_index(drop=True)

In [19]:
show_df(song_df)

(170653, 18)


,song_description,song_name,song_artists,song_acousticness,song_danceability,song_duration_ms,song_energy,song_explicit,song_instrumentalness,song_key,song_liveness,song_loudness,song_mode,song_popularity,song_speechiness,song_tempo,song_valence,song_year
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve - Sergei Rachmaninoff, James Levine, Berliner Philharmoniker","Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","Sergei Rachmaninoff, James Levine, Berliner Philharmoniker",0.982,0.279,831667,0.211,0,0.878000,10,0.665,-20.096,1,4,0.0366,80.954,0.0594,1921
1,Clancy Lowered the Boom - Dennis Day,Clancy Lowered the Boom,Dennis Day,0.732,0.819,180533,0.341,0,0.000000,7,0.160,-12.441,1,5,0.4150,60.936,0.9630,1921
2,Gati Bali - KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,Gati Bali,KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,0.961,0.328,500062,0.166,0,0.913000,3,0.101,-14.850,1,5,0.0339,110.339,0.0394,1921
3,Danny Boy - Frank Parker,Danny Boy,Frank Parker,0.967,0.275,210000,0.309,0,0.000028,5,0.381,-9.316,1,3,0.0354,100.109,0.1650,1921
4,When Irish Eyes Are Smiling - Phil Regan,When Irish Eyes Are Smiling,Phil Regan,0.957,0.418,166693,0.193,0,0.000002,3,0.229,-10.096,1,2,0.0380,101.665,0.2530,1921


## 3. Create Vector Embeddings

We will create vector embeddings from this data in three steps:
1. Encoding the categorical song_description as numeric values,
1. Scaling the numeric column values,
1. Joining these two sets of encodings together into one vector embedding.

### Embed Categorical Song Metadata

To embed the song descriptions as numeric vectors, we must perform natural language processing on them.
This involves tokenising the descriptions to break them up into their individual sub-parts and then using a Word2Vec model to turn these tokenised song descriptions into vectors.

The length of the vectors we turn these desciptions into is configurable, however, in this case we chose to set this to `15` as there are also 15 numeric columns which describe the song. We do not want to bias the final embedding vectors in favour of either the categorical columns or the numeric columns, so it made sense to keep the number of values representing each the same for both.

In [20]:
# tokenise the descriptions
tokenised_song_descs = [word_tokenize(v.lower()) for v in song_data["song_description"]]

In [21]:
# create embedding model
embedding_dim = 15
word2Vec_model = Word2Vec(
    sentences=tokenised_song_descs,
    vector_size=embedding_dim,
    window=5,
    min_count=1,
    sg=1,
)

In [22]:
# function to create embedding vector from tokens
def get_embedding(song_desc_tokens, model, embedding_dim):
    vectors = [model.wv[token] for token in song_desc_tokens if token in model.wv]

    # Average of word vectors OR zeros if no valid tokens found
    return sum(vectors) / len(vectors) if vectors else [0] * embedding_dim

In [23]:
# embed song descriptions as vectors
categorical_embeddings = [
    get_embedding(song_desc_tokens, word2Vec_model, embedding_dim)
    for song_desc_tokens in tokenised_song_descs
]

In [24]:
show_embeddings(categorical_embeddings)

Num Embeddings: 157685
Embedding Size: 15


[-1.3084614,
 0.66102946,
 1.6010187,
 -0.8913263,
 -0.5413131,
 -0.3870546,
 -1.2124491,
 -0.5503847,
 -0.42727256,
 1.4809988,
 0.26244432,
 0.27446768,
 0.2161151,
 -1.0858109,
 -0.842732]

### Embed Numeric Song Metadata

There are 15 numeric columns in our data which we will use to make up the other half of our final embedding vectors.
First, however, we will scale these values to make them more uniform.

The standard scaled score of a sample `x` is calculated as:

    z = (x - u) / s

where `u` is the mean of the training samples and `s` is the standard deviation of the training samples.

In [25]:
# extract numeric columns
numeric_cols = list(
    song_data.drop(columns=["song_name", "song_artists", "song_description"]).columns
)
numeric_cols

['song_acousticness',
 'song_danceability',
 'song_duration_ms',
 'song_energy',
 'song_explicit',
 'song_instrumentalness',
 'song_key',
 'song_liveness',
 'song_loudness',
 'song_mode',
 'song_popularity',
 'song_speechiness',
 'song_tempo',
 'song_valence',
 'song_year']

In [26]:
# scale these columns
numeric_embeddings = StandardScaler().fit_transform(song_data[numeric_cols])

In [27]:
show_embeddings(numeric_embeddings)

Num Embeddings: 157685
Embedding Size: 15


[1.2703070294949106,
 -1.461259048884883,
 4.752569009266444,
 -1.007676175100162,
 -0.3092011481361043,
 2.262496351074803,
 1.3649563314116429,
 2.6110012104955738,
 -1.5078176079821606,
 0.6453499264358126,
 -1.2499471942272533,
 -0.38364744367670833,
 -1.1655450558051375,
 -1.7786347004763523,
 -2.142666230649]

### Merge Categorical & Numeric Embeddings

This leaves us with two sets of vectors: one representing the categorical column and one representing the numeric columns.
Both sets have 15 values each, so when we join these together, the resulting vector will have 30 values.

In [28]:
row_embeddings = [
    np.concatenate([cat_row, num_row])
    for cat_row, num_row in zip(categorical_embeddings, numeric_embeddings)
]

In [29]:
show_embeddings(row_embeddings)

Num Embeddings: 157685
Embedding Size: 30


[-1.3084614276885986,
 0.6610294580459595,
 1.6010186672210693,
 -0.8913263082504272,
 -0.541313111782074,
 -0.38705459237098694,
 -1.212449073791504,
 -0.5503847002983093,
 -0.4272725582122803,
 1.4809987545013428,
 0.26244431734085083,
 0.27446767687797546,
 0.2161151021718979,
 -1.085810899734497,
 -0.8427320122718811,
 1.2703070294949106,
 -1.461259048884883,
 4.752569009266444,
 -1.007676175100162,
 -0.3092011481361043,
 2.262496351074803,
 1.3649563314116429,
 2.6110012104955738,
 -1.5078176079821606,
 0.6453499264358126,
 -1.2499471942272533,
 -0.38364744367670833,
 -1.1655450558051375,
 -1.7786347004763523,
 -2.142666230649]

### Create DataFrame With Embeddings

We can take these defined embeddings and create a Pandas DataFrame containing them.
This will be the table we insert into our vector database.

To enable proper filtering of the data once inserted into the KDB.AI vector database, we will pair these embedding vectors with three song description columns: `song_name`, `song_artists`, and `song_year`.

In [30]:
embedded_song_df = song_data[["song_name", "song_artists", "song_year"]]

In [31]:
embedded_song_df["song_embeddings"] = row_embeddings

In [32]:
show_df(embedded_song_df)

(157685, 4)


,song_name,song_artists,song_year,song_embeddings
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","Sergei Rachmaninoff, James Levine, Berliner Philharmoniker",1921,"[-1.3084614276885986, 0.6610294580459595, 1.6010186672210693, -0.8913263082504272, -0.541313111782074, -0.38705459237098694, -1.212449073791504, -0.5503847002983093, -0.4272725582122803, 1.4809987545013428, 0.26244431734085083, 0.27446767687797546, 0.2161151021718979, -1.085810899734497, -0.8427320122718811, 1.2703070294949106, -1.461259048884883, 4.752569009266444, -1.007676175100162, -0.3092011481361043, 2.262496351074803, 1.3649563314116429, 2.6110012104955738, -1.5078176079821606, 0.6453499264358126, -1.2499471942272533, -0.38364744367670833, -1.1655450558051375, -1.7786347004763523, -2.142666230649]"
1,Clancy Lowered the Boom,Dennis Day,1921,"[-0.46706071496009827, 0.22206126153469086, 1.6218688488006592, -0.6445634961128235, 0.15803197026252747, 0.29785123467445374, -0.553114116191864, -0.07888539880514145, -0.35507890582084656, 0.12263522297143936, 0.6071826219558716, 0.6136294603347778, 0.5297210812568665, -0.7411094307899475, 0.0877525731921196, 0.6055353575765545, 1.600008527700499, -0.3958314054754836, -0.5219481676317469, -0.3092011481361043, -0.5349944602375606, 0.5117210944216953, -0.26640400859153673, -0.1646257835500665, 0.6453499264358126, -1.2044048830284118, 1.872793548131166, -1.8164526201212032, 1.6541851866630062, -2.142666230649]"
2,Gati Bali,KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,1921,"[-0.5912865400314331, 0.19827693700790405, 0.5054091811180115, 0.2651362419128418, 0.5384843349456787, 0.21282942593097687, 0.3593631386756897, -0.42768657207489014, -0.22806212306022644, 0.5242754220962524, 0.7951330542564392, 0.06683247536420822, -0.1490078866481781, -0.2776865065097809, -1.2925686836242676, 1.2144662090537686, -1.183477361379913, 2.1306274127125904, -1.1758127930699978, -0.3092011481361043, 2.374013638541697, -0.6259258882315683, -0.6025761034947833, -0.5873232499193564, 0.6453499264358126, -1.2044048830284118, -0.3997478418740689, -0.21005905433508043, -1.854615663007104, -2.142666230649]"
3,Danny Boy,Frank Parker,1921,"[-0.4566471576690674, 0.5354787111282349, 1.7839219570159912, -1.2448246479034424, 0.024612929672002792, 0.26332181692123413, -0.8040102124214172, 0.16238971054553986, -0.35203710198402405, 0.37662094831466675, 0.7859536409378052, 0.6951225996017456, 0.438914954662323, -0.7189780473709106, 0.012237541377544403, 1.2304207291798093, -1.4839351050077376, -0.16284109162119192, -0.6415119848547415, -0.3092011481361043, -0.5349062022700511, -0.05710239690493648, 0.9928168892663868, 0.3837053008849818, 0.6453499264358126, -1.295489505426095, -0.39080317620886856, -0.5426988976237884, -1.3774552183139837, -2.142666230649]"
4,When Irish Eyes Are Smiling,Phil Regan,1921,"[-0.4916699230670929, 0.2079925537109375, 1.5473459959030151, -0.6245953440666199, 0.6297972798347473, 0.08845526725053787, -1.0048013925552368, 0.13564133644104004, -0.448259562253952, -0.01485042180866003, 1.0761586427688599, 0.3285246193408966, 0.7505322098731995, -0.7707496881484985, -0.13520577549934387, 1.203829862303075, -0.6732660986156829, -0.5052618172494476, -1.0749308222880962, -0.3092011481361043, -0.5349891074077622, -0.6259258882315683, 0.12674640748175162, 0.24684186220999385, 0.6453499264358126, -1.3410318166249366, -0.3752990890558546, -0.4921038246856425, -1.0431389831786766, -2.142666230649]"


## 4. Create KDB.AI Vector Database

Now that our data has been read in and our embeddings have been defined, we can set about creating a KDB.AI vector database and ingesting the data into it.

Once we have the database set up, we can then query our data and start deriving some insights

### Connect To KDB.AI Session

To use KDB.AI, you will need two session details - a URL endpoint and an API key. To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI session using `kdbai.Session`. Enter the session URL endpoint and API key details from your KDB.AI Cloud portal below.

In [ ]:
KDBAI_ENDPOINT = input('KDB.AI endpoint: ')
KDBAI_API_KEY = getpass('KDB.AI API key: ')
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

We can check if there are any tables defined in this KDB.AI session - it should not contain a table called `songs`.

In [31]:
session.list()

[]

### Define Vector DB Schema

The next step is to define the schema for the table in KDB.AI which will store our embeddings.

As mentioned above, our table will have four columns:
- Song Name
- Song Artists
- Song Year
- Song Embeddings

When defining the schema, we must supply the types of these columns. We can use the `.dtypes()` function on the defined Pandas DataFrame to help with this.

In [37]:
embedded_song_df.dtypes

song_name          object
song_artists       object
song_year           int64
song_embeddings    object
dtype: object

In [38]:
schema = {
    "columns": [
        {
            "name": "song_name",
            "pytype": "str",
        },
        {
            "name": "song_artists",
            "pytype": "str",
        },
        {
            "name": "song_year",
            "pytype": "int64",
        },
        {
            "name": "song_embeddings",
            "vectorIndex": {
                "dims": len(numeric_cols) + embedding_dim,
                "metric": "L2",
                "type": "flat",
            },
        },
    ]
}

### Create Vector DB Table

Using this defined schema, we will create a table in the KDB.AI vector database to house the emebedding data.

In [47]:
songs = session.create_table("songs", schema)

### Add Embedding Data to Vector DB Table

When adding larger amounts of data, you should insert data into an index in chunks.

It is a good idea to first get an idea of how large your dataset to insert is.

In [40]:
embedded_song_df.memory_usage(deep=True).sum() / (1024**2)

79.63715362548828

This dataset is 80MB which exceeds the insert limit of <10MB at a time. As such, we'll insert this data in chunks, inserting 10,000 rows at a time.

In [41]:
chunk_size = 10_000

In [48]:
for i in range((len(embedded_song_df) // chunk_size) + 1):
    index = i * chunk_size
    songs.insert(
        embedded_song_df.iloc[index : index + chunk_size].reset_index(drop=True)
    )

In [49]:
songs.query()

,song_name,song_artists,song_year,song_embeddings
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","Sergei Rachmaninoff, James Levine, Berliner Philharmoniker",1921,"[-1.3084614276885986, 0.6610294580459595, 1.6010186672210693, -0.8913263082504272, -0.541313111782074, -0.38705459237098694, -1.212449073791504, -0.5503847002983093, -0.4272725582122803, 1.4809987545013428, 0.26244431734085083, 0.27446767687797546, 0.2161151021718979, -1.085810899734497, -0.8427320122718811, 1.2703070294949106, -1.461259048884883, 4.752569009266444, -1.007676175100162, -0.3092011481361043, 2.262496351074803, 1.3649563314116429, 2.6110012104955738, -1.5078176079821606, 0.6453499264358126, -1.2499471942272533, -0.38364744367670833, -1.1655450558051375, -1.7786347004763523, -2.142666230649]"
1,Clancy Lowered the Boom,Dennis Day,1921,"[-0.46706071496009827, 0.22206126153469086, 1.6218688488006592, -0.6445634961128235, 0.15803197026252747, 0.29785123467445374, -0.553114116191864, -0.07888539880514145, -0.35507890582084656, 0.12263522297143936, 0.6071826219558716, 0.6136294603347778, 0.5297210812568665, -0.7411094307899475, 0.0877525731921196, 0.6055353575765545, 1.600008527700499, -0.3958314054754836, -0.5219481676317469, -0.3092011481361043, -0.5349944602375606, 0.5117210944216953, -0.26640400859153673, -0.1646257835500665, 0.6453499264358126, -1.2044048830284118, 1.872793548131166, -1.8164526201212032, 1.6541851866630062, -2.142666230649]"
2,Gati Bali,KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,1921,"[-0.5912865400314331, 0.19827693700790405, 0.5054091811180115, 0.2651362419128418, 0.5384843349456787, 0.21282942593097687, 0.3593631386756897, -0.42768657207489014, -0.22806212306022644, 0.5242754220962524, 0.7951330542564392, 0.06683247536420822, -0.1490078866481781, -0.2776865065097809, -1.2925686836242676, 1.2144662090537686, -1.183477361379913, 2.1306274127125904, -1.1758127930699978, -0.3092011481361043, 2.374013638541697, -0.6259258882315683, -0.6025761034947833, -0.5873232499193564, 0.6453499264358126, -1.2044048830284118, -0.3997478418740689, -0.21005905433508043, -1.854615663007104, -2.142666230649]"
3,Danny Boy,Frank Parker,1921,"[-0.4566471576690674, 0.5354787111282349, 1.7839219570159912, -1.2448246479034424, 0.024612929672002792, 0.26332181692123413, -0.8040102124214172, 0.16238971054553986, -0.35203710198402405, 0.37662094831466675, 0.7859536409378052, 0.6951225996017456, 0.438914954662323, -0.7189780473709106, 0.012237541377544403, 1.2304207291798093, -1.4839351050077376, -0.16284109162119192, -0.6415119848547415, -0.3092011481361043, -0.5349062022700511, -0.05710239690493648, 0.9928168892663868, 0.3837053008849818, 0.6453499264358126, -1.295489505426095, -0.39080317620886856, -0.5426988976237884, -1.3774552183139837, -2.142666230649]"
4,When Irish Eyes Are Smiling,Phil Regan,1921,"[-0.4916699230670929, 0.2079925537109375, 1.5473459959030151, -0.6245953440666199, 0.6297972798347473, 0.08845526725053787, -1.0048013925552368, 0.13564133644104004, -0.448259562253952, -0.01485042180866003, 1.0761586427688599, 0.3285246193408966, 0.7505322098731995, -0.7707496881484985, -0.13520577549934387, 1.203829862303075, -0.6732660986156829, -0.5052618172494476, -1.0749308222880962, -0.3092011481361043, -0.5349891074077622, -0.6259258882315683, 0.12674640748175162, 0.24684186220999385, 0.6453499264358126, -1.3410318166249366, -0.3752990890558546, -0.4921038246856425, -1.0431389831786766, -2.142666230649]"
...,...,...,...,...
157680,China,"Anuel AA, Daddy Yankee, KAROL G, Ozuna, J Balvin",2020,"[-0.28655603528022766, 0.9745289087295532, 1.3392137289047241, -0.9253003001213074, -0.2074325680732727, 0.7460130453109741, -0.7828342318534851, -0.4283033609390259, 0.2643398344516754, 1.1300415992736816, 0.7959958910942078, 0.37394899129867554, -0.08583994954824448, -0.07435858994722366, -0.35595691204071045, -1.1159573640232203, 1.4129310646869482, 0.5623251501011589, 1.222936289966328, -0.3092011481361043, -0.5340736460639054, 0.511

In [50]:
show_df(songs.query())

(157685, 4)


,song_name,song_artists,song_year,song_embeddings
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve","Sergei Rachmaninoff, James Levine, Berliner Philharmoniker",1921,"[-1.3084614276885986, 0.6610294580459595, 1.6010186672210693, -0.8913263082504272, -0.541313111782074, -0.38705459237098694, -1.212449073791504, -0.5503847002983093, -0.4272725582122803, 1.4809987545013428, 0.26244431734085083, 0.27446767687797546, 0.2161151021718979, -1.085810899734497, -0.8427320122718811, 1.2703070294949106, -1.461259048884883, 4.752569009266444, -1.007676175100162, -0.3092011481361043, 2.262496351074803, 1.3649563314116429, 2.6110012104955738, -1.5078176079821606, 0.6453499264358126, -1.2499471942272533, -0.38364744367670833, -1.1655450558051375, -1.7786347004763523, -2.142666230649]"
1,Clancy Lowered the Boom,Dennis Day,1921,"[-0.46706071496009827, 0.22206126153469086, 1.6218688488006592, -0.6445634961128235, 0.15803197026252747, 0.29785123467445374, -0.553114116191864, -0.07888539880514145, -0.35507890582084656, 0.12263522297143936, 0.6071826219558716, 0.6136294603347778, 0.5297210812568665, -0.7411094307899475, 0.0877525731921196, 0.6055353575765545, 1.600008527700499, -0.3958314054754836, -0.5219481676317469, -0.3092011481361043, -0.5349944602375606, 0.5117210944216953, -0.26640400859153673, -0.1646257835500665, 0.6453499264358126, -1.2044048830284118, 1.872793548131166, -1.8164526201212032, 1.6541851866630062, -2.142666230649]"
2,Gati Bali,KHP Kridhamardawa Karaton Ngayogyakarta Hadiningrat,1921,"[-0.5912865400314331, 0.19827693700790405, 0.5054091811180115, 0.2651362419128418, 0.5384843349456787, 0.21282942593097687, 0.3593631386756897, -0.42768657207489014, -0.22806212306022644, 0.5242754220962524, 0.7951330542564392, 0.06683247536420822, -0.1490078866481781, -0.2776865065097809, -1.2925686836242676, 1.2144662090537686, -1.183477361379913, 2.1306274127125904, -1.1758127930699978, -0.3092011481361043, 2.374013638541697, -0.6259258882315683, -0.6025761034947833, -0.5873232499193564, 0.6453499264358126, -1.2044048830284118, -0.3997478418740689, -0.21005905433508043, -1.854615663007104, -2.142666230649]"
3,Danny Boy,Frank Parker,1921,"[-0.4566471576690674, 0.5354787111282349, 1.7839219570159912, -1.2448246479034424, 0.024612929672002792, 0.26332181692123413, -0.8040102124214172, 0.16238971054553986, -0.35203710198402405, 0.37662094831466675, 0.7859536409378052, 0.6951225996017456, 0.438914954662323, -0.7189780473709106, 0.012237541377544403, 1.2304207291798093, -1.4839351050077376, -0.16284109162119192, -0.6415119848547415, -0.3092011481361043, -0.5349062022700511, -0.05710239690493648, 0.9928168892663868, 0.3837053008849818, 0.6453499264358126, -1.295489505426095, -0.39080317620886856, -0.5426988976237884, -1.3774552183139837, -2.142666230649]"
4,When Irish Eyes Are Smiling,Phil Regan,1921,"[-0.4916699230670929, 0.2079925537109375, 1.5473459959030151, -0.6245953440666199, 0.6297972798347473, 0.08845526725053787, -1.0048013925552368, 0.13564133644104004, -0.448259562253952, -0.01485042180866003, 1.0761586427688599, 0.3285246193408966, 0.7505322098731995, -0.7707496881484985, -0.13520577549934387, 1.203829862303075, -0.6732660986156829, -0.5052618172494476, -1.0749308222880962, -0.3092011481361043, -0.5349891074077622, -0.6259258882315683, 0.12674640748175162, 0.24684186220999385, 0.6453499264358126, -1.3410318166249366, -0.3752990890558546, -0.4921038246856425, -1.0431389831786766, -2.142666230649]"


## 5. Query Songs From Vector Database

Now that the data has been inserted into the database, we can perform some queries on the data.

### Find Songs By Artist

We can query the database to find songs by particular artists using KDB.AI's `.query()` function.

Here, we want to return all songs in the dataset by the DJ `Calvin Harris`, sorted by the year they were produced. This returns 32 songs to us.

In [51]:
songs.query(filter=[("like", "song_artists", "*Calvin Harris*")], sort_by="song_year")

,song_name,song_artists,song_year,song_embeddings
0,Flashback,Calvin Harris,2009,"[-0.4910580515861511, 0.540467381477356, 1.3442914485931396, -1.1832348108291626, 0.5024847984313965, 0.4400637745857239, -0.8165419101715088, -0.14912326633930206, 0.13948674499988556, 0.15553173422813416, 0.41881316900253296, 0.602066695690155, 0.3767136335372925, -0.3130016028881073, -0.17774870991706848, -1.3351724705550174, -1.5406252453148743, -0.011030115749796763, 1.7460279903169285, -0.3092011481361043, -0.5313940449564866, 1.0805445857483271, -0.7039975151774578, 1.0250333372402145, -1.549546934208044, 0.7539144985217788, -0.307319630000332, 0.36420802995835755, -1.2520866301382436, 1.247808968405853]"
1,You Used To Hold Me,Calvin Harris,2009,"[-0.4107224941253662, 0.6976003050804138, 1.8933608531951904, -1.1152050495147705, 0.3909798264503479, 0.1973215937614441, -1.1214869022369385, 0.12395014613866806, -0.28961804509162903, 0.09116952866315842, 1.361146092414856, 0.2691359519958496, 0.8772494196891785, -0.22174984216690063, -0.1768137663602829, -1.279491195315136, 0.46053670752705134, 0.004253770518922344, 1.8244917453695186, -0.3092011481361043, -0.5349904137531296, 1.6493680770749588, 0.6224578016611152, 0.958707209266951, -1.549546934208044, 0.5717452537264123, -0.35562082459241384, 0.3955535635781341, -1.2824790151505443, 1.247808968405853]"
2,I'm Not Alone - Radio Edit,Calvin Harris,2009,"[-0.5333942770957947, 0.632325291633606, 1.6398407220840454, -1.0604794025421143, 0.2829623520374298, 0.18867354094982147, -0.9835246801376343, 0.1709904968738556, -0.19129319489002228, 0.24490037560462952, 1.0491188764572144, 0.36916524171829224, 0.8066525459289551, -0.44579797983169556, -0.11600116640329361, -1.3273547556932577, 0.3074733286977821, -0.1491385696844665, 0.7932538218211915, -0.3092011481361043, -0.14309084999676247, 0.5117210944216953, 0.4914076629700189, 0.8665875870818629, 0.6453499264358126, 0.3895760089310457, -0.411077751716656, 0.46191895166730823, -0.36310936852844955, 1.247808968405853]"
3,We Found Love,"Rihanna, Calvin Harris",2011,"[-0.3598034977912903, 0.7455580830574036, 1.6835709810256958, -1.104311227798462, 0.2709399461746216, 0.22481369972229004, -1.127227544784546, 0.005336357280611992, -0.034981757402420044, 0.25602656602859497, 0.934143602848053, 0.438564270734787, 0.7639918327331543, -0.5546166896820068, -0.23971381783485413, -1.2744389306085564, 1.1181423350898372, -0.12151213480453658, 1.0660087798611477, -0.3092011481361043, -0.5305974929031516, -1.1947493795582, -0.5626912786757541, 1.2313812909348116, 0.6453499264358126, 1.9835569008905032, -0.37351015592281456, 0.36375280436636925, 0.2751307167298641, 1.3248652229298272]"
4,Dance Wiv Me - Radio Edit,"Dizzee Rascal, Calvin Harris, Chrome",2011,"[-0.38744503259658813, 0.5095685720443726, 1.3607453107833862, -0.9676647186279297, 0.1452278345823288, 0.26811453700065613, -0.56062251329422, 0.058428164571523666, -0.23659648001194, 0.222921684384346, 0.5812807083129883, 0.40284159779548645, 0.28188979625701904, -0.3537159562110901, -0.19545702636241913, -1.2143435714671371, 1.9344803555126058, -0.20954668716662583, 0.9912813940967762, -0.3092011481361043, -0.5349944602375606, 1.6493680770749588, -0.30059100129356187, 1.2671763441267319, 0.6453499264358126, 1.5281337889020867, -0.3329610049072396, -0.1561798539118754, 1.00454795702508, 1.3248652229298272]"
5,Feel So Close - Radio Edit,Calvin Harris,2012,"[-0.44324296712875366, 0.6264983415603638, 1.615656852722168, -1.2315382957458496, 0.36009278893470764, 0.4174306392669678, -0.9566606879234314, 0.1460820585489273, -0.3323456048965454, 0.22914759814739227, 0.9515199661254883, 0.47497859597206116, 0.6885043978691101, -0.3836240768432617, -0.10300761461257935, -1.3383314655399736, 0.9650789562605678, -0.19120286091549893, 1.6563551273996828, -0.3092011481361043, -0.5125954164977816, 0.5117210944216953, -0.015699395443352974, 1.5196718418873827, 0.6453499264358126, 2.2112684568847114, -0.4170408621601229, 0.3621

### Find Specific Song

We can filter this query further by looking for the song `We Found Love` by `Calvin Harris` in our dataset.

This will only return one song as he only produced one song with this name.

In [52]:
songs.query(
    filter=[
        ("like", "song_artists", "*Calvin Harris*"),
        ("like", "song_name", "*We Found Love*"),
    ]
)

,song_name,song_artists,song_year,song_embeddings
0,We Found Love,"Rihanna, Calvin Harris",2011,"[-0.3598034977912903, 0.7455580830574036, 1.6835709810256958, -1.104311227798462, 0.2709399461746216, 0.22481369972229004, -1.127227544784546, 0.005336357280611992, -0.034981757402420044, 0.25602656602859497, 0.934143602848053, 0.438564270734787, 0.7639918327331543, -0.5546166896820068, -0.23971381783485413, -1.2744389306085564, 1.1181423350898372, -0.12151213480453658, 1.0660087798611477, -0.3092011481361043, -0.5305974929031516, -1.1947493795582, -0.5626912786757541, 1.2313812909348116, 0.6453499264358126, 1.9835569008905032, -0.37351015592281456, 0.36375280436636925, 0.2751307167298641, 1.3248652229298272]"


### Find Similar Songs

We can then copy and paste the vector associated with this song below and save it as the variable `my_vec`.

We will then use KDB.AI's `.search()` function to find similar songs in the dataset to this song using this vector.
We will pull out the 5 songs most similar to this song from the dataset.

<div class="alert alert-block alert-warning">
Note that the most similar song will be "We Found Love" by "Calvin Harris" as this is the vector we are using for the search.
</div>

In [53]:
my_vec = [-0.19870510697364807, 1.3078361749649048, 1.0181931257247925, -0.3232698142528534, 0.98063725233078, 0.26473715901374817, -1.0277972221374512, 0.0383748784661293, 0.1495048850774765, 0.23416249454021454, 1.1922948360443115, 0.47670742869377136, 1.1959521770477295, -0.08822440356016159, 0.042070869356393814, -1.2744389306085564, 1.1181423350898372, -0.12151213480453658, 1.0660087798611477, -0.3092011481361043, -0.5305974929031516, -1.1947493795582, -0.5626912786757541, 1.2313812909348116, 0.6453499264358126, 1.9835569008905032, -0.37351015592281456, 0.36375280436636925, 0.2751307167298641, 1.3248652229298272]

In [54]:
songs.search(vectors=[my_vec], n=5)[0]

,song_name,song_artists,song_year,song_embeddings,__nn_distance
0,We Found Love,"Rihanna, Calvin Harris",2011,"[-0.3598034977912903, 0.7455580830574036, 1.6835709810256958, -1.104311227798462, 0.2709399461746216, 0.22481369972229004, -1.127227544784546, 0.005336357280611992, -0.034981757402420044, 0.25602656602859497, 0.934143602848053, 0.438564270734787, 0.7639918327331543, -0.5546166896820068, -0.23971381783485413, -1.2744389306085564, 1.1181423350898372, -0.12151213480453658, 1.0660087798611477, -0.3092011481361043, -0.5305974929031516, -1.1947493795582, -0.5626912786757541, 1.2313812909348116, 0.6453499264358126, 1.9835569008905032, -0.37351015592281456, 0.36375280436636925, 0.2751307167298641, 1.3248652229298272]",2.497229
1,Try Everything,Shakira,2016,"[-0.45326852798461914, 0.7631129026412964, 1.4206933975219727, -0.5168099999427795, 0.10055868327617645, 0.3728799819946289, -0.9679427742958069, 0.3327452540397644, -0.3841341435909271, 0.3373335599899292, 1.3592381477355957, 0.3151901066303253, 0.5810573101043701, -0.08865176141262054, -0.06225752830505371, -1.272577569927185, 0.9537409281991405, -0.26679207598479776, 0.47939880161083126, -0.3092011481361043, -0.5349944602375606, -1.1947493795582, -0.696020550213652, 1.159089320762895, 0.6453499264358126, 1.7103030336974534, -0.41882979529316294, -0.0430888104164571, -0.19595125096079602, 1.517505859239762]",3.092355
2,Since U Been Gone,Kelly Clarkson,2004,"[-0.11152409762144089, 0.6174209713935852, 1.5025031566619873, -0.8987154364585876, 0.333455353975296, -0.06713759899139404, -1.147297739982605, 0.03241661936044693, -0.1582876592874527, 0.39395079016685486, 1.5894933938980103, 0.5742592215538025, 0.9644389152526855, -0.6342738270759583, -0.2583732306957245, -1.3365286047657308, 0.7099733248784531, -0.3292006193469291, 0.9725995476556833, -0.3092011481361043, -0.438452351373364, -1.479161125221516, -0.528504285973729, 1.0697771537301144, 0.6453499264358126, 1.8469299672939783, -0.40272939709580236, 0.4617563710987411, -0.4694827160715017, 1.0551683320959184]",3.318148
3,Bad At Love,Halsey,2017,"[-0.5517164468765259, 0.34906429052352905, 1.7956831455230713, -0.675864577293396, 0.1774449646472931, 0.12092027813196182, -0.9815996289253235, 0.11402924358844757, -0.10657869279384613, 0.4558851718902588, 1.2332789897918701, 0.42635613679885864, 0.6847960352897644, -0.6413012146949768, 0.21150214970111847, -1.1803072618649173, 0.7836705072777309, -0.3899329165171471, 1.0099632405378691, -0.3092011481361043, -0.5349944602375606, -1.479161125221516, -0.6692407392637322, 1.3973720768149898, 0.6453499264358126, 1.9380145896916616, -0.4253892167809766, 0.051533080489714826, 0.32071929424831513, 1.556033986501749]",3.634671
4,I'm Not Mad,Halsey,2020,"[-0.4158092439174652, 0.6362892985343933, 1.4361778497695923, -0.6862751841545105, 0.19531361758708954, -0.10460829734802246, -1.0953339338302612, 0.28047657012939453, -0.005529905203729868, 0.4606150686740875, 1.5008271932601929, 0.23351790010929108, 1.011540412902832, -0.6577786803245544, -0.09148161858320236, -0.8888713608959099, 1.3789169805026662, -0.45170100732482105, 0.7596264982272247, -0.3092011481361043, -0.5349944602375606, -1.479161125221516, -0.5797847750267666, 1.0080131403793504, 0.6453499264358126, 1.9380145896916616, -0.22264346170310204, 1.0768311781025728, 0.7766050694328249, 1.67161836828771]",3.682489


### Automate This Process

We can define a function to automate this process and find songs which are the most similar to any input song.
This will allow us to use the KDB.AI vector database in a more production-like setting to perform similarity search and music recommendation.

In [55]:
def find_similar_songs(
    vectorDB_song_tab,
    song_name: str,
    song_artists: list[str] = None,
    song_year: int = None,
    n_similar: int = 5,
    exact: bool = False,
) -> None:

    # create filter list
    filter_list = [("like", "song_name", f"{song_name}" if exact else f"*{song_name}*")]
    if song_artists:
        if type(song_artists) == str:
            song_artists = list(song_artists)
        for artist in song_artists:
            filter_list.append(("like", "song_artists", f"*{artist}*"))
    if song_year:
        filter_list.append(("like", "song_year", f"{song_year}"))

    # find songs liks this in vector DB
    resulting_song = vectorDB_song_tab.query(filter=filter_list, sort_by="song_year")

    # quality check
    if resulting_song.empty:
        print(
            "Song Not Found! Please double check the values entered or try another song"
        )
        return

    # find vectors associated with these songs
    resulting_vectors = [v.tolist() for v in resulting_song["song_embeddings"]]

    # search for similar songs to selected songs
    similar_songs = songs.search(vectors=resulting_vectors, n=n_similar + 1)

    # process similar song table
    for i, similar_df in enumerate(similar_songs):
        name = resulting_song.loc[i, "song_name"]
        artists = resulting_song.loc[i, "song_artists"]
        year = resulting_song.loc[i, "song_year"]
        print(f"Songs Similar To '{name}' By '{artists}' ({year})")
        for j, song in similar_df[1:].iterrows():
            print(
                f"   {j}. {song['song_name']} - {song['song_artists']} ({song['song_year']})"
            )
        print()

In [56]:
# song by multiple artists
find_similar_songs(songs, song_name="Let's Go", song_artists=["Calvin Harris", "Ne-Yo"])

Songs Similar To 'Let's Go (feat. Ne-Yo)' By 'Calvin Harris, Ne-Yo' (2012)
   1. I Cry - Flo Rida (2012)
   2. Mmm Yeah (feat. Pitbull) - Austin Mahone, Pitbull (2014)
   3. Too Much (feat. Usher) - Marshmello, Imanbek, Usher (2020)
   4. All Around The World - Justin Bieber, Ludacris (2012)
   5. Play Hard (feat. Ne-Yo & Akon) - David Guetta (2011)



In [57]:
# specify different number of similar songs
find_similar_songs(songs, song_name="Californication", song_artists="Red Hot Chili Peppers", n_similar=8)

Songs Similar To 'Californication' By 'Red Hot Chili Peppers' (1999)
   1. Police Station - Red Hot Chili Peppers (2011)
   2. Charlie - Red Hot Chili Peppers (2006)
   3. Dark Necessities - Red Hot Chili Peppers (2016)
   4. Especially in Michigan - Red Hot Chili Peppers (2006)
   5. Don't Forget Me - Red Hot Chili Peppers (2002)
   6. Cabron - Red Hot Chili Peppers (2002)
   7. Face Down - The Red Jumpsuit Apparatus (2006)
   8. Look Around - Red Hot Chili Peppers (2011)



In [58]:
# all songs with a given name by any artist
find_similar_songs(songs, song_name="Love Me", exact=True)

Songs Similar To 'Love Me' By 'Elvis Presley' (1956)
   1. Don't - Elvis Presley (1959)
   2. Without Him - Elvis Presley (1967)
   3. Harbor Lights - Elvis Presley (1959)
   4. His Hand in Mine - Elvis Presley (1960)
   5. You Go To My Head - Billie Holiday (1956)

Songs Similar To 'Love Me' By 'Buddy Holly' (1958)
   1. A Love That's Worth Having - Willie Hutch (1969)
   2. Johnny be good - Radio Version - Jonny Bombastic (1955)
   3. Lonely Weekends - Wanda Jackson (1961)
   4. Midnight Shift - Buddy Holly (1958)
   5. Rock & Roll Guitar - Johnny Knight (1959)

Songs Similar To 'Love Me' By 'Sarah Vaughan' (1958)
   1. Summer Is Gone - Carmen McRae (1956)
   2. Make the World Go Away - Ray Price (1956)
   3. A Pretty Girl Is Like A Melody - Irving Berlin, Ethel Merman, Dan Dailey (1954)
   4. I'll Come Back For More - Dinah Washington (1961)
   5. I'm Confessin' (That I Love You) - Judy Garland (1958)

Songs Similar To 'Love Me' By 'Bo Diddley' (1960)
   1. Time To Go - Lesley Gore 

These result are good quality as the top results include plenty of similar songs.

With that, we have built a recommendation system which is able to recommend music based both on user numerical and categorical song data.